In [105]:
## 问题
    ### 我们没有数据
    ### 我们不知道哪些特征会影响人找工作，权重如何
    ### 人最终面试、上岗受到很多因素影响。如【有其他更优质工作机会、突发事件】等。这些不应该作为【匹配失败】的数据。那么什么样的数据应该作为失败数据呢？上岗成功的一定算是匹配成功，但不匹配的数据仍需要由逻辑得出

## 解题思路
    ### 特征选取
        ### 根据现实情况选择部分显著特征，每种特征根据情况折算得分，得分高于 80 的认为最终可以上岗成功（这里把特征总结成枚举项，便于随机生成）
        ### 特征之间可能相互影响，如家庭背景影响薪资权重，婚姻状况影响上班耗时权重（暂不考虑）
        ### 后续补充权重计算【可能应该按折算成薪资后的数额进行】
    ### 数据来源
        ### 没有足够外部数据，就由逻辑生成随机数据源
        ### 数据源足够大时(约为 2000 条即可)，可让 AI 掌握由人类总结的逻辑
        ### 此时输出结果应与现有逻辑匹配保持一致
    ### 后续业务数据集成
        ### 最终上岗的业务数据可以直接与生成的随机数据源混合，训练后查看结果，如果仍获得较高拟合度，则认为我们总结的特征及权重符合实际。如果发现拟合度较低，分析原因
        ### 后续持续更换特征，迭代找出更合适的模型
        ### 着重分析最终上岗数据中的较低分值的数据，发掘其中隐藏的特征

In [106]:
# 按照以下逻辑生成数据作为基础数据，表达逻辑匹配推荐算法
# 基准逻辑算法
dict_raws_base = {
    # 性别
    # 0 岗位有硬性要求 人员不符合
    # 1 岗位有期望要求 人员不符合
    # 2 岗位有期望要求 人员符合
    # 3 岗位无要求
    # 4 岗位有硬性要求 人员符合
    'gender': {
        'high': 5,
        'score': [-10000, 0, 100, 100, 120],
    },
    # 时间匹配度（PS：时间匹配度的概念是由人的时间与岗位需要之间时间矩阵的交集，情况复杂，需要独立算法单独计算，此处不展开）
    # 1 / 7 是 14.25%
    # 0 <15%
    # 1 >=15% <60%
    # 2 >=60% <90%
    # 3 >90%
    'time_match_rate': {
        'high': 4,
        'score': [-10000, 0, 60, 100],
    },
    # 职位类别 
    # 0 非常不满意
    # 1 不满意
    # 2 满意
    'job_categories': {
        'high': 3,
        'score': [0, 60, 100],
    },
    # 缴金
    # 0 公司不缴纳+用户在乎
    # 1 公司缴纳+用户不在乎
    # 2 公司缴纳+用户在乎
    # 3 公司不缴纳+用户不在乎
    'insurance': {
        'high': 4,
        'score': [0, 60, 100, 100],
    },
    # 薪资+福利(包吃住等)
    # 0 薪资低于预期+无福利
    # 1 薪资低于预期+有福利
    # 2 薪资达到预期+无福利
    # 3 薪资达到预期+有福利
    'benefits': {
        'high': 4,
        'score': [0, 20, 80, 100],
    },
    # 上班耗时
    # 0 超过2小时
    # 1 超过1小时
    # 2 40分钟以内
    # 3 20分钟以内
    'go_work_time': {
        'high': 4,
        'score': [0, 20, 80, 100],
    },
}
# 假定特征相同，只是具体规则有出入
columns = dict_raws_base.keys()
# 得分模型
def get_score(d, score_raws):
    count = 0
    for key in columns:
        score = score_raws[key]['score'][int(d[key])]
        # 暂时不设置权重，认为各特征权重相等 weight = score_raws[key]['weight']
        count += (score / len(columns))
    return count
def get_score_rate(d, score_key):
    # 得分 0 总分 < 60 不考虑
    # 得分 1 总分 60 ~ 80 可以考虑
    # 得分 2 总分 > 80 有较强上岗可能性
    rate = 0
    count = d[score_key]
    if count < 60:
        rate = 0
    if count >= 60 and count < 80:
        rate = 1
    if count >= 80:
        rate = 2
    return rate

In [107]:
import numpy as np
import pandas as pd

def get_df(dict_raws, size):
    df_raws = {}
    for key in dict_raws:
        df_raws[key] = np.random.randint(dict_raws[key]['high'], size=(size))
    df=pd.DataFrame(df_raws, columns=columns)
    return df

df_mock = get_df(dict_raws_base, 20000)
df_mock["score"] = df_mock.apply(lambda d:get_score(d, dict_raws_base),axis =1)
df_mock["score_rate"] = df_mock.apply(lambda d:get_score_rate(d, 'score'),axis =1)
# 模拟数据
df_mock

,gender,time_match_rate,job_categories,insurance,benefits,go_work_time,score,score_rate
0,0,3,1,1,3,3,-1596.666667,0
1,4,0,0,3,3,1,-1610.000000,0
2,0,2,2,0,0,3,-1623.333333,0
3,1,1,1,1,0,0,20.000000,0
4,1,2,2,1,1,2,53.333333,0
...,...,...,...,...,...,...,...,...
19995,2,3,1,1,1,3,73.333333,1
19996,4,3,1,0,2,3,76.666667,1
19997,1,2,1,3,0,0,36.666667,0
19998,4,3,1,0,3,2,76.666667,1


In [108]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

def get_modal(df_data):
    data = df_data[columns].values
    target = df_data["score_rate"].values
    x_train, x_test, y_train, y_test = train_test_split(data,target,test_size=0.3)
    r_lf = RandomForestClassifier(random_state=0)
    r_lf = r_lf.fit(x_train,y_train)
    r_score = r_lf.score(x_test,y_test)
    # 得到 1.0 评分认为是机器学习模型可以正确的学会基准逻辑，认为随机模拟的数据量已经足够
    print("随机森林模型评分:{}".format(r_score))
    importances = r_lf.feature_importances_
    indices = np.argsort(importances)[::-1]
    feat_labels = df_data.columns[0:]
    print('特征重要性分析:')
    for f in range(x_train.shape[1]):
        print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))
    return r_lf
df_mock_modal = get_modal(df_mock)

随机森林模型评分:0.9993333333333333
特征重要性分析:
 1) time_match_rate                0.280778
 2) gender                         0.232471
 3) job_categories                 0.140745
 4) insurance                      0.116617
 5) go_work_time                   0.115246
 6) benefits                       0.114143


In [109]:
# 结果预测测试
print(df_mock_modal.predict([
    [0,0,0,0,0,0],
    [1,1,1,1,1,1],
    [3,3,3,3,3,3],
]))

[0 0 2]


In [110]:
## 模型可以转化成代码供程序使用
import m2cgen as m2c
code = m2c.export_to_java(df_mock_modal)

In [111]:
## 以上是模型-策略的相互转化过程，基本可以得出结论，数据与模型本质上是一回事
## 后续展开业务漏损数据对模型进行修正的过程

# 假设后续出现一些漏损的业务数据(PS: 漏损数据指评分较低，但仍上岗成功的数据，或者评分极高，但用户或商家明确表示不合适的数据，两种数据同样重要，如果仅有一种，会造成模型向其中一个方向偏移)
# 鉴别漏损数据的过程，可能会形成新的关键特征。如果特征调整，需要使用现有已上岗数据对策略进行回测（不展开讨论）
# 漏损数据符合的得分模型假定如下（实际上我们不知道具体的漏损数据模型，但是有漏损数据，这里定义模型来生成数据）
dict_raws_busi = {
    # 性别
    # 0 岗位有硬性要求 人员不符合
    # 1 岗位有期望要求 人员不符合
    # 2 岗位有期望要求 人员符合
    # 3 岗位无要求
    # 4 岗位有硬性要求 人员符合
    'gender': {
        'high': 5,
        'score': [0, 0, 100, 100, 120],
    },
    # 时间匹配度（PS：时间匹配度的概念是由人的时间与岗位需要之间时间矩阵的交集，情况复杂，需要独立算法单独计算，此处不展开）
    # 1 / 7 是 14.25%
    # 0 <15%
    # 1 >=15% <60%
    # 2 >=60% <90%
    # 3 >90%
    'time_match_rate': {
        'high': 4,
        'score': [0, 0, 60, 100],
    },
    # 职位类别 
    # 0 非常不满意
    # 1 不满意
    # 2 满意
    'job_categories': {
        'high': 3,
        'score': [0, 60, 100],
    },
    # 缴金
    # 0 公司不缴纳+用户在乎
    # 1 公司缴纳+用户不在乎
    # 2 公司缴纳+用户在乎
    # 3 公司不缴纳+用户不在乎
    'insurance': {
        'high': 4,
        'score': [0, 30, 60, 100],
    },
    # 薪资+福利(包吃住等)
    # 0 薪资低于预期+无福利
    # 1 薪资低于预期+有福利
    # 2 薪资达到预期+无福利
    # 3 薪资达到预期+有福利
    'benefits': {
        'high': 4,
        'score': [0, 40, 100, 100],
    },
    # 上班耗时
    # 0 超过2小时
    # 1 超过1小时
    # 2 40分钟以内
    # 3 20分钟以内
    'go_work_time': {
        'high': 4,
        'score': [0, 50, 80, 200],
    },
}

In [112]:
df_busi = get_df(dict_raws_base, 60000)
# 基准模型评分
df_busi["score"] = df_busi.apply(lambda d:get_score(d, dict_raws_base),axis =1)
df_busi["score_rate"] = df_busi.apply(lambda d:get_score_rate(d, 'score'),axis =1)
# 真实模型评分
df_busi["busi_score"] = df_busi.apply(lambda d:get_score(d, dict_raws_busi),axis =1)
df_busi["busi_score_rate"] = df_busi.apply(lambda d:get_score_rate(d, 'busi_score'),axis =1)
# 过滤出基准模型评分等级为 0 但漏损模型评分等级为 2 的数据。即已上岗但未能推荐，反之假定为明确表示不合适的岗，这两者之间应该大体均等（或者标准模型评分为 1 但已上岗应该纠正为 2？）
# 样本量级比较小，本身作为训练集很容易出现过拟合
# df_busi_no 的数据在真实业务中难以排查
df_busi_yes = df_busi.query('score_rate==0 & busi_score_rate==2')
df_busi_no = df_busi.query('score_rate==2 & busi_score_rate==0')
# df_busi = pd.concat([df_busi_yes, df_busi_no])
df_busi = df_busi_yes
df_busi

,gender,time_match_rate,job_categories,insurance,benefits,go_work_time,score,score_rate,busi_score,busi_score_rate
35,3,0,2,2,1,3,-1596.666667,0,83.333333,2
52,3,0,1,3,1,3,-1603.333333,0,83.333333,2
92,0,2,1,2,2,3,-1600.000000,0,80.000000,2
97,2,0,2,3,0,3,-1600.000000,0,83.333333,2
118,0,3,1,2,3,3,-1590.000000,0,86.666667,2
...,...,...,...,...,...,...,...,...,...,...
59610,0,0,2,3,2,3,-3270.000000,0,83.333333,2
59710,2,0,2,3,1,3,-1596.666667,0,90.000000,2
59773,3,0,1,3,3,3,-1590.000000,0,93.333333,2
59855,2,0,1,3,1,3,-1603.333333,0,83.333333,2


In [113]:
df_busi['score'] = df_busi['busi_score']
df_busi['score_rate'] = df_busi['busi_score_rate']
df_busi = df_busi.drop(columns=['busi_score', 'busi_score_rate'])

In [114]:
# 直接用业务数据训练模型，发现找不到规律？此处如果已经找到规律了，直接使用即可，真实的业务数据由于特征数量不足，不推荐岗位(评级为0)的数据难以界定，应该难以找到规律，如果能找到规律，全篇都可以理解为废话了。
df_busi_modal = get_modal(df_busi)
df_busi_modal.predict([[0,0,0,0,0,0]]) # 只有上岗数据训练出来的模型，得到的结论永远都是 2

随机森林模型评分:1.0
特征重要性分析:
 1) go_work_time                   0.000000
 2) benefits                       0.000000
 3) insurance                      0.000000
 4) job_categories                 0.000000
 5) time_match_rate                0.000000
 6) gender                         0.000000


array([2])

In [115]:
# 合并漏损数据与模拟数据
df_combine = pd.concat([df_mock, df_busi])
df_combine

,gender,time_match_rate,job_categories,insurance,benefits,go_work_time,score,score_rate
0,0,3,1,1,3,3,-1596.666667,0
1,4,0,0,3,3,1,-1610.000000,0
2,0,2,2,0,0,3,-1623.333333,0
3,1,1,1,1,0,0,20.000000,0
4,1,2,2,1,1,2,53.333333,0
...,...,...,...,...,...,...,...,...
59610,0,0,2,3,2,3,83.333333,2
59710,2,0,2,3,1,3,90.000000,2
59773,3,0,1,3,3,3,93.333333,2
59855,2,0,1,3,1,3,83.333333,2


In [116]:
# 重新训练生成模型，如果评分较高说明数据仍有规律可寻，我们距离真实模型正在靠近？
df_combine_modal = get_modal(df_combine)

随机森林模型评分:0.9712274257728803
特征重要性分析:
 1) time_match_rate                0.229604
 2) gender                         0.194701
 3) go_work_time                   0.166938
 4) insurance                      0.146562
 5) job_categories                 0.138232
 6) benefits                       0.123964


In [117]:
# 这些是被认为不能上岗，未进行推荐的数据，现在的模型下有可能已经推荐上岗了
df_combine_modal.predict(df_busi_yes[columns].values)

array([2, 2, 2, ..., 2, 2, 2])